# DLML - GAN

## Random Seeds

In [0]:
# Set Numpy seed.
from numpy.random import seed
seed(1)

# Set TensorFlow seed.
from tensorflow import set_random_seed
set_random_seed(2)

## Imports

In [0]:
import keras
import keras.backend as K

from keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.layers import BatchNormalization, Dropout, ZeroPadding2D
from keras.layers import Activation, LeakyReLU, ReLU

from keras.models import Sequential, Model, load_model
from keras.initializers import RandomNormal, glorot_normal
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.datasets import mnist, fashion_mnist, cifar10

import os
import time
import datetime
import numpy as np
import pandas as pd
from IPython.display import HTML

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.animation as animation
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## Functions

In [0]:
# Return random noise.
def sample_noise(size):
    return np.random.normal(0, 1, size=size)
    #return np.random.uniform(-1, 1, size=size)

In [0]:
# Return datetime.
def get_datetime(seconds):
    return str(datetime.timedelta(seconds=seconds))

In [0]:
# Return moving average of argument.
def ema(data, window_size=100):
    cumsum_vec = np.cumsum(np.insert(data, 0, 0)) 
    ma_vec = (cumsum_vec[window_size:] - cumsum_vec[:-window_size]) / window_size
    return ma_vec

In [0]:
# Return the norm of all gradients of trinable parameters (only works in training).
def get_gradient_norm(model):
    with K.name_scope('grad_norm'):
        grads = K.gradients(model.total_loss, model.trainable_weights)
        norm = K.sqrt(sum([K.sum(K.square(g)) for g in grads]))
    return norm

# Data

In [0]:
# Data dimensions.
x_dim = 28
y_dim = 28
channels = 1
img_shape = (x_dim, y_dim, channels)

In [0]:
# Load data.
(x_train, _), (x_test, _) = mnist.load_data()

In [0]:
# Add channel dimension.
x_train = x_train.reshape(-1, x_dim, y_dim, channels)

# Normalize data to interval (-1, 1).
x_train = x_train / 255 * 2 - 1

In [0]:
# Print data shape.
print(x_train.shape)

In [0]:
plt.imshow(x_train[6].reshape(x_dim, y_dim), cmap='gray_r')
plt.show()

# GAN - Aufgaben

## Hyper Parameters

In [0]:
iterations = 10000
batch_size = 128
latent_dim = 100

init = glorot_normal() #RandomNormal(mean=0, stddev=0.02)
optimizer = Adam(lr=0.0002, beta_1=0.5)

## Generator

In [0]:
# Build generator model.
# ---------- Aufgabe ----------
# Erstellen Sie eine Input layer z mit
# dem shape latent_dim.
z = ...

x = Dense(128 * 7 * 7, kernel_initializer=init)(z)
x = LeakyReLU(0.2)(x)
x = Reshape((7, 7, 128))(x)

x = Conv2D(128, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2DTranspose(128, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2D(64, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2DTranspose(64, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2D(channels, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = Activation("tanh")(x)

generator = Model(z, x)
generator.name = "generator"
generator.summary()

## Discriminator

In [0]:
# Build discriminator model.
# ---------- Aufgabe ----------
# Erstellen Sie eine Input layer img_input mit
# dem shape eines MNIST Bildes.
# Schauen Sie ggf. oben nach, welchen shape die
# MNIST Daten haben.
img_input = ...

x = Conv2D(32, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(img_input)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(64, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(128, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(256, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(1, kernel_initializer=init)(x)
x = Activation("sigmoid")(x)

discriminator = Model(img_input, x)
discriminator.name = "discriminator"

# ---------- Aufgabe ----------
# Fügen Sie die richtige loss function (loss=...)
# des Diskriminators. Der Diskriminator soll
# binäre Klassifikation machen.
discriminator.compile(loss=...,
    optimizer=optimizer,
    metrics=['accuracy'])

discriminator.metrics_names.append("grad_norm")
discriminator.metrics_tensors.append(get_gradient_norm(discriminator))

discriminator.summary()

## Combined Model

In [0]:
# For the combined model we will only train the generator.
# ---------- Aufgabe ----------
# Frieren Sie die Gewichte des Diskriminators ein.
...

# Combined model (stacked generator and discriminator).
combined = Model(z, discriminator(generator(z)))
combined.name = "combined"
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

combined.metrics_names.append("grad_norm")
combined.metrics_tensors.append(get_gradient_norm(combined))

combined.summary()

## Training

In [0]:
# Loss history.
losses = {
    'd_loss_real' : [],
    'd_loss_fake' : [],
    'd_acc_real' : [],
    'd_acc_fake' : [],
    'd_grad_norm_real' : [],
    'd_grad_norm_fake' : [],
    'd_loss' : [],
    'd_acc' : [],
    'd_grad_norm' : [],
    'g_loss' : [],
    'g_grad_norm' : []
}

In [0]:
# Savely create path.
save_path_image = "images/"
if not os.path.exists(save_path_image):
    os.mkdir(save_path_image)

In [0]:
# Adversarial ground truths.
# ---------- Aufgabe ----------
# Erstellen Sie einen Vektor valid vom shape (batch_size, 1) der aus 1 besteht.
valid = ...
# Erstellen Sie einen Vektor fake vom shape (batch_size, 1) der aus 0 besteht.
fake = ...

# Start time measurement.
start = time.time()

# Save interval settings.
save_interval = 50
use_static_noise = True
r, c = 5, 5
static_noise = np.random.normal(0, 1, (r * c, latent_dim))
save_paths = []

for iteration in range(iterations):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random half of images.
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train[idx]

    # Sample noise and generate a batch of new images.
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    # Train the discriminator (real classified as ones and generated as zeros).
    # ---------- Aufgabe ----------
    # Sie brauchen die Variablen: imgs, gen_imgs, fake, valid
    # Geben Sie der Funktion train_on_batch die richtigen Werte um:
    # 1. Auf echten Daten zu trainieren.
    d_loss_real, d_acc_real, d_grad_norm_real = discriminator.train_on_batch(..., ...)
    # 2. Auf den generierten Daten zu trainieren.
    d_loss_fake, d_acc_fake, d_grad_norm_fake = discriminator.train_on_batch(..., ...)

    # Calculate discriminator metrics (average of real and fake batches) to record.
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    d_acc = 0.5 * np.add(d_acc_real, d_acc_fake)
    d_grad_norm = 0.5 * np.add(d_grad_norm_real, d_grad_norm_fake)

    # Log discriminator metrics.
    losses['d_loss_real'].append(d_loss_real.tolist())
    losses['d_loss_fake'].append(d_loss_fake.tolist())
    losses['d_acc_real'].append(d_acc_real.tolist())
    losses['d_acc_fake'].append(d_acc_fake.tolist())
    losses['d_grad_norm_real'].append(d_grad_norm_real.tolist())
    losses['d_grad_norm_fake'].append(d_grad_norm_fake.tolist())
    losses['d_loss'].append(d_loss.tolist())
    losses['d_acc'].append(d_acc.tolist())
    losses['d_grad_norm'].append(d_grad_norm.tolist())

    # ---------------------
    #  Train Generator
    # ---------------------

    # Sample novel noise.
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Train the generator (wants discriminator to mistake images as real).
    # ---------- Aufgabe ----------
    # Geben Sie der Funktion train_on_batch die richtigen Werte um
    # den Generator zu trainiere.
    g_loss, g_grad_norm = combined.train_on_batch(..., ...)
    
    # Log generator metrics.
    losses['g_loss'].append(g_loss)
    losses['g_grad_norm'].append(g_grad_norm)

    # Print progress.
    print("Iteration: {:5d} [D loss: {:1.5f}, acc.: {:5.2f}%, grad norm: {:5.2f}] [G loss: {:1.5f}, grad norm: {:5.2f}]".format(
        iteration + 1, d_loss, 100 * d_acc, d_grad_norm, g_loss, g_grad_norm))

    # Save generated image samples.
    if (iteration + 1) % save_interval == 0 or (iteration + 1) == 1:
        if use_static_noise:
            noise = static_noise # Use static noise.
        else:
            noise = np.random.normal(0, 1, (r * c, latent_dim)) # Use new radom noise every save iteration.
        gen_imgs = generator.predict(noise)

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray_r')
                axs[i,j].axis('off')
                cnt += 1
        fig.suptitle("Iteration: {:d}".format(iteration + 1))
        iter_save_path = "{:s}iteration_{:d}.png".format(save_path_image, iteration + 1)
        save_paths.append(iter_save_path)
        fig.savefig(iter_save_path)
        plt.show()

    # Finish time measurement.
    if (iteration + 1) == iterations:
        end = time.time()
        training_time = end - start
        print('Finished training in {:s}'.format(get_datetime(training_time)))

In [0]:
# Savely create path.
save_path_training = "training/"
if not os.path.exists(save_path_training):
    os.mkdir(save_path_training)

In [0]:
# Create animation of generated images during trainig.
fig = plt.figure()
ims = []
for save_path in save_paths:
    im = plt.imshow(mpimg.imread(save_path) , cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('{:s}training.mp4'.format(save_path_training))
HTML(ani.to_html5_video())

In [0]:
# Plot generator and discriminator loss.
d_loss = np.array(losses['d_loss'])
g_loss = np.array(losses['g_loss'])

plt.plot(d_loss, label="D loss")
plt.plot(g_loss, label="G loss")

plt.legend(loc="best")
plt.savefig("{:s}D and G loss".format(save_path_training))
plt.show()

In [0]:
# Plot generator and discriminator loss (real and fake).
d_loss = np.array(losses['d_loss'])
g_loss = np.array(losses['g_loss'])
d_loss_real = np.array(losses['d_loss_real'])
d_loss_fake = np.array(losses['d_loss_fake'])

plt.plot(d_loss, label="D loss")
plt.plot(g_loss, label="G loss")
plt.plot(d_loss_real, label="D loss (real)")
plt.plot(d_loss_fake, label="D loss (fake)")

plt.legend(loc="best")
plt.savefig("{:s}D and G loss (real and fake)".format(save_path_training))
plt.show()

In [0]:
# Plot discriminator accuracy.
d_acc = np.array(losses['d_acc'])

d_acc_ema = ema(d_acc, window_size=100)

plt.plot(d_acc, label="D acc")
plt.plot(d_acc_ema, label="D acc (ema)")

plt.legend(loc="best")
plt.savefig("{:s}D acc".format(save_path_training))
plt.show()

In [0]:
# Plot discriminator accuracy (real and fake).
d_acc = np.array(losses['d_acc'])
d_acc_ema = ema(d_acc, window_size=100)
d_acc_real = np.array(losses['d_acc_real'])
d_acc_real_ema = ema(d_acc_real, window_size=100)
d_acc_fake = np.array(losses['d_acc_fake'])
d_acc_fake_ema = ema(d_acc_fake, window_size=100)

plt.plot(d_acc, label="D acc")
plt.plot(d_acc_ema, label="D acc (ema)")
plt.plot(d_acc_real, label="D acc (real)")
plt.plot(d_acc_real_ema, label="D acc (real, ema)")
plt.plot(d_acc_fake, label="D acc (fake)")
plt.plot(d_acc_fake_ema, label="D acc (real, ema)")

plt.legend(loc="best")
plt.savefig("{:s}D acc (real and fake)".format(save_path_training))
plt.show()

In [0]:
# Plot generator and discriminator gradient norm.
d_grad_norm = np.array(losses['d_grad_norm'])
g_grad_norm = np.array(losses['g_grad_norm'])

plt.plot(d_grad_norm, label="D grad norm")
plt.plot(g_grad_norm, label="G grad norm")

plt.legend(loc="best")
plt.savefig("{:s}D and G grad norm".format(save_path_training))
plt.show()

In [0]:
# Plot generator and discriminator gradient norm (real and fake).
d_grad_norm = np.array(losses['d_grad_norm'])
g_grad_norm = np.array(losses['g_grad_norm'])
d_grad_norm_real = np.array(losses['d_grad_norm_real'])
d_grad_norm_fake = np.array(losses['d_grad_norm_fake'])

plt.plot(d_grad_norm, label="D grad norm")
plt.plot(g_grad_norm, label="G grad norm")
plt.plot(d_grad_norm_real, label="D grad norm (real)")
plt.plot(d_grad_norm_fake, label="D grad norm (fake)")

plt.legend(loc="best")
plt.savefig("{:s}D and G grad norm (real and fake)".format(save_path_training))
plt.show()

## Generate from Latent Space

In [0]:
# Savely create path.
save_path_gen_image = "gen_images/"
if not os.path.exists(save_path_gen_image):
    os.mkdir(save_path_gen_image)

In [0]:
# Generate images from random noise z.
r, c = 5, 5
noise = np.random.normal(0, 1, (r * c, latent_dim))
gen_imgs = generator.predict(noise)

fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray_r')
        axs[i,j].axis('off')
        cnt += 1

fig.suptitle("Generated Images")
fig.savefig("{:s}gen_image.png".format(save_path_gen_image))
plt.show()
plt.close()

In [0]:
# Visualize latent space travel from point A to point B.
intermediate_steps = 100

# Points in z.
point_a = np.random.normal(0, 1, (1, latent_dim))
point_b = np.random.normal(0, 1, (1, latent_dim))

points_travel = np.array(
    [point_a + (point_b - point_a) * x / intermediate_steps for x in range(0, 1 + intermediate_steps)]).reshape(-1, latent_dim)

# Images generated from z.
image_a = generator.predict(point_a)
image_b = generator.predict(point_b)
images_travel = generator.predict(points_travel)

# Show origin and destination image.
fig, axs = plt.subplots(1, 2)
axs[0].imshow(image_a.reshape(x_dim, y_dim), cmap='gray_r')
axs[0].axis('off')
axs[0].title.set_text("Origin image")
axs[1].imshow(image_b.reshape(x_dim, y_dim), cmap='gray_r')
axs[1].axis('off')
axs[1].title.set_text("Destination image")
plt.show()

# Create animation.
fig = plt.figure()
ims = []
for image_travel in images_travel:
    im = plt.imshow(image_travel.reshape(x_dim, y_dim), cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('latent_travel.mp4')
HTML(ani.to_html5_video())

In [0]:
# Travel freely through latent space.
n_travel = 1000
points = []
images = []

# Initial points.
#point_prev = np.full((1, latent_dim), 0)
point_prev = np.random.normal(0, 1, (1, latent_dim))
point_curr = point_prev + 0.1 * np.random.uniform(-0.1, 0.1, (1, latent_dim))

# Travel.
print("Calculating travel points...")
for i in range(n_travel):
    direction = (point_prev - point_curr) + 0.25 * np.random.uniform(-0.1, 0.1, (1, latent_dim))
    point = point_prev + direction
    point = point.clip(min=-3, max=3)
    points.append(point)
    point_curr = point_prev
    point_prev = point

points = np.array(points).reshape(-1, latent_dim)

print("Generate images...")
images = generator.predict(points)

# Create animation.
print("Creating animation...")
fig = plt.figure()
ims = []
for image in images:
    im = plt.imshow(image.reshape(x_dim, y_dim), cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('latent_travel_n.mp4')
HTML(ani.to_html5_video())

# GAN - Lösungen

## Hyper Parameters

In [0]:
iterations = 10000
batch_size = 128
latent_dim = 100

init = glorot_normal() #RandomNormal(mean=0, stddev=0.02)
optimizer = Adam(lr=0.0002, beta_1=0.5)

## Generator

In [0]:
# Build generator model.
z = Input(shape=(latent_dim,))
x = Dense(128 * 7 * 7, kernel_initializer=init)(z)
x = LeakyReLU(0.2)(x)
x = Reshape((7, 7, 128))(x)

x = Conv2D(128, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2DTranspose(128, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2D(64, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2DTranspose(64, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2D(channels, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = Activation("tanh")(x)

generator = Model(z, x)
generator.name = "generator"
generator.summary()

## Discriminator

In [0]:
# Build discriminator model.
img_input = Input(shape=img_shape)
x = Conv2D(32, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(img_input)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(64, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(128, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(256, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(1, kernel_initializer=init)(x)
x = Activation("sigmoid")(x)

discriminator = Model(img_input, x)
discriminator.name = "discriminator"
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

discriminator.metrics_names.append("grad_norm")
discriminator.metrics_tensors.append(get_gradient_norm(discriminator))

discriminator.summary()

## Combined Model

In [0]:
# For the combined model we will only train the generator.
discriminator.trainable = False

# Combined model (stacked generator and discriminator).
combined = Model(z, discriminator(generator(z)))
combined.name = "combined"
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

combined.metrics_names.append("grad_norm")
combined.metrics_tensors.append(get_gradient_norm(combined))

combined.summary()

## Training

In [0]:
# Loss history.
losses = {
    'd_loss_real' : [],
    'd_loss_fake' : [],
    'd_acc_real' : [],
    'd_acc_fake' : [],
    'd_grad_norm_real' : [],
    'd_grad_norm_fake' : [],
    'd_loss' : [],
    'd_acc' : [],
    'd_grad_norm' : [],
    'g_loss' : [],
    'g_grad_norm' : []
}

In [0]:
# Savely create path.
save_path_image = "images/"
if not os.path.exists(save_path_image):
    os.mkdir(save_path_image)

In [0]:
# Adversarial ground truths.
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))
#valid = np.full((batch_size, 1), 0.9)
#fake = np.full((batch_size, 1), 0.1)

# Start time measurement.
start = time.time()

# Save interval settings.
save_interval = 50
use_static_noise = True
r, c = 5, 5
static_noise = np.random.normal(0, 1, (r * c, latent_dim))
save_paths = []

for iteration in range(iterations):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random half of images.
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train[idx]

    # Sample noise and generate a batch of new images.
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    # Train the discriminator (real classified as ones and generated as zeros).
    d_loss_real, d_acc_real, d_grad_norm_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake, d_acc_fake, d_grad_norm_fake = discriminator.train_on_batch(gen_imgs, fake)

    # Calculate discriminator metrics (average of real and fake batches) to record.
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    d_acc = 0.5 * np.add(d_acc_real, d_acc_fake)
    d_grad_norm = 0.5 * np.add(d_grad_norm_real, d_grad_norm_fake)

    # Log discriminator metrics.
    losses['d_loss_real'].append(d_loss_real.tolist())
    losses['d_loss_fake'].append(d_loss_fake.tolist())
    losses['d_acc_real'].append(d_acc_real.tolist())
    losses['d_acc_fake'].append(d_acc_fake.tolist())
    losses['d_grad_norm_real'].append(d_grad_norm_real.tolist())
    losses['d_grad_norm_fake'].append(d_grad_norm_fake.tolist())
    losses['d_loss'].append(d_loss.tolist())
    losses['d_acc'].append(d_acc.tolist())
    losses['d_grad_norm'].append(d_grad_norm.tolist())

    # ---------------------
    #  Train Generator
    # ---------------------

    # Sample novel noise.
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Train the generator (wants discriminator to mistake images as real).
    g_loss, g_grad_norm = combined.train_on_batch(noise, valid)
    
    # Log generator metrics.
    losses['g_loss'].append(g_loss)
    losses['g_grad_norm'].append(g_grad_norm)

    # Print progress.
    print("Iteration: {:5d} [D loss: {:1.5f}, acc.: {:5.2f}%, grad norm: {:5.2f}] [G loss: {:1.5f}, grad norm: {:5.2f}]".format(
        iteration + 1, d_loss, 100 * d_acc, d_grad_norm, g_loss, g_grad_norm))

    # Save generated image samples.
    if (iteration + 1) % save_interval == 0 or (iteration + 1) == 1:
        if use_static_noise:
            noise = static_noise # Use static noise.
        else:
            noise = np.random.normal(0, 1, (r * c, latent_dim)) # Use new radom noise every save iteration.
        gen_imgs = generator.predict(noise)

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray_r')
                axs[i,j].axis('off')
                cnt += 1
        fig.suptitle("Iteration: {:d}".format(iteration + 1))
        iter_save_path = "{:s}iteration_{:d}.png".format(save_path_image, iteration + 1)
        save_paths.append(iter_save_path)
        fig.savefig(iter_save_path)
        plt.show()

    # Finish time measurement.
    if (iteration + 1) == iterations:
        end = time.time()
        training_time = end - start
        print('Finished training in {:s}'.format(get_datetime(training_time)))

In [0]:
# Savely create path.
save_path_training = "training/"
if not os.path.exists(save_path_training):
    os.mkdir(save_path_training)

In [0]:
# Create animation of generated images during trainig.
fig = plt.figure()
ims = []
for save_path in save_paths:
    im = plt.imshow(mpimg.imread(save_path) , cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('{:s}training.mp4'.format(save_path_training))
HTML(ani.to_html5_video())

In [0]:
# Plot generator and discriminator loss.
d_loss = np.array(losses['d_loss'])
g_loss = np.array(losses['g_loss'])

plt.plot(d_loss, label="D loss")
plt.plot(g_loss, label="G loss")

plt.legend(loc="best")
plt.savefig("{:s}D and G loss".format(save_path_training))
plt.show()

In [0]:
# Plot generator and discriminator loss (real and fake).
d_loss = np.array(losses['d_loss'])
g_loss = np.array(losses['g_loss'])
d_loss_real = np.array(losses['d_loss_real'])
d_loss_fake = np.array(losses['d_loss_fake'])

plt.plot(d_loss, label="D loss")
plt.plot(g_loss, label="G loss")
plt.plot(d_loss_real, label="D loss (real)")
plt.plot(d_loss_fake, label="D loss (fake)")

plt.legend(loc="best")
plt.savefig("{:s}D and G loss (real and fake)".format(save_path_training))
plt.show()

In [0]:
# Plot discriminator accuracy.
d_acc = np.array(losses['d_acc'])

d_acc_ema = ema(d_acc, window_size=100)

plt.plot(d_acc, label="D acc")
plt.plot(d_acc_ema, label="D acc (ema)")

plt.legend(loc="best")
plt.savefig("{:s}D acc".format(save_path_training))
plt.show()

In [0]:
# Plot discriminator accuracy (real and fake).
d_acc = np.array(losses['d_acc'])
d_acc_ema = ema(d_acc, window_size=100)
d_acc_real = np.array(losses['d_acc_real'])
d_acc_real_ema = ema(d_acc_real, window_size=100)
d_acc_fake = np.array(losses['d_acc_fake'])
d_acc_fake_ema = ema(d_acc_fake, window_size=100)

plt.plot(d_acc, label="D acc")
plt.plot(d_acc_ema, label="D acc (ema)")
plt.plot(d_acc_real, label="D acc (real)")
plt.plot(d_acc_real_ema, label="D acc (real, ema)")
plt.plot(d_acc_fake, label="D acc (fake)")
plt.plot(d_acc_fake_ema, label="D acc (real, ema)")

plt.legend(loc="best")
plt.savefig("{:s}D acc (real and fake)".format(save_path_training))
plt.show()

In [0]:
# Plot generator and discriminator gradient norm.
d_grad_norm = np.array(losses['d_grad_norm'])
g_grad_norm = np.array(losses['g_grad_norm'])

plt.plot(d_grad_norm, label="D grad norm")
plt.plot(g_grad_norm, label="G grad norm")

plt.legend(loc="best")
plt.savefig("{:s}D and G grad norm".format(save_path_training))
plt.show()

In [0]:
# Plot generator and discriminator gradient norm (real and fake).
d_grad_norm = np.array(losses['d_grad_norm'])
g_grad_norm = np.array(losses['g_grad_norm'])
d_grad_norm_real = np.array(losses['d_grad_norm_real'])
d_grad_norm_fake = np.array(losses['d_grad_norm_fake'])

plt.plot(d_grad_norm, label="D grad norm")
plt.plot(g_grad_norm, label="G grad norm")
plt.plot(d_grad_norm_real, label="D grad norm (real)")
plt.plot(d_grad_norm_fake, label="D grad norm (fake)")

plt.legend(loc="best")
plt.savefig("{:s}D and G grad norm (real and fake)".format(save_path_training))
plt.show()

## Generate from Latent Space

In [0]:
# Savely create path.
save_path_gen_image = "gen_images/"
if not os.path.exists(save_path_gen_image):
    os.mkdir(save_path_gen_image)

In [0]:
# Generate images from random noise z.
r, c = 5, 5
noise = np.random.normal(0, 1, (r * c, latent_dim))
gen_imgs = generator.predict(noise)

fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray_r')
        axs[i,j].axis('off')
        cnt += 1

fig.suptitle("Generated Images")
fig.savefig("{:s}gen_image.png".format(save_path_gen_image))
plt.show()
plt.close()

In [0]:
# Visualize latent space travel from point A to point B.
intermediate_steps = 100

# Points in z.
point_a = np.random.normal(0, 1, (1, latent_dim))
point_b = np.random.normal(0, 1, (1, latent_dim))

points_travel = np.array(
    [point_a + (point_b - point_a) * x / intermediate_steps for x in range(0, 1 + intermediate_steps)]).reshape(-1, latent_dim)

# Images generated from z.
image_a = generator.predict(point_a)
image_b = generator.predict(point_b)
images_travel = generator.predict(points_travel)

# Show origin and destination image.
fig, axs = plt.subplots(1, 2)
axs[0].imshow(image_a.reshape(x_dim, y_dim), cmap='gray_r')
axs[0].axis('off')
axs[0].title.set_text("Origin image")
axs[1].imshow(image_b.reshape(x_dim, y_dim), cmap='gray_r')
axs[1].axis('off')
axs[1].title.set_text("Destination image")
plt.show()

# Create animation.
fig = plt.figure()
ims = []
for image_travel in images_travel:
    im = plt.imshow(image_travel.reshape(x_dim, y_dim), cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('latent_travel.mp4')
HTML(ani.to_html5_video())

In [0]:
# Travel freely through latent space.
n_travel = 1000
points = []
images = []

# Initial points.
#point_prev = np.full((1, latent_dim), 0)
point_prev = np.random.normal(0, 1, (1, latent_dim))
point_curr = point_prev + 0.1 * np.random.uniform(-0.1, 0.1, (1, latent_dim))

# Travel.
print("Calculating travel points...")
for i in range(n_travel):
    direction = (point_prev - point_curr) + 0.25 * np.random.uniform(-0.1, 0.1, (1, latent_dim))
    point = point_prev + direction
    point = point.clip(min=-3, max=3)
    points.append(point)
    point_curr = point_prev
    point_prev = point

points = np.array(points).reshape(-1, latent_dim)

print("Generate images...")
images = generator.predict(points)

# Create animation.
print("Creating animation...")
fig = plt.figure()
ims = []
for image in images:
    im = plt.imshow(image.reshape(x_dim, y_dim), cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('latent_travel_n.mp4')
HTML(ani.to_html5_video())